<a href="https://colab.research.google.com/github/eccho03/datamining_project_clothes/blob/main/item_base_collaborate_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import re

# 정보 입력

In [3]:
##### 사용자 정보(예시) #####
user_gender = 'W'  # 성별 ('female' 또는 'male')
user_height = 170  # 예시 키 (cm)
user_weight = 65   # 예시 몸무게 (kg)
min_price = 50000    # 최소 가격 (예시)
max_price = 100000    # 최대 가격 (예시)
select_clothes = 50 # 원하는 옷 (예시)

In [4]:
user_gender = input("W/M(성별) :")
user_height = int(input("키(cm) :"))
user_weight = int(input("몸무게(kg) :"))
min_price = int(input("최소 가격 :"))
max_price = int(input("최대 가격 :"))

W/M(성별) :W
키(cm) :158
몸무게(kg) :51
최소 가격 :50000
최대 가격 :300000


# 데이터 업로드 및 데이터 정리

In [5]:
if user_gender == 'W':
    data = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/products_info_women.csv')
elif user_gender == 'M':
    data = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/products_info_men.csv')
else:
    raise ValueError("성별은 'female' 또는 'male'이어야 합니다.")

In [6]:
size_data = pd.read_csv('https://raw.githubusercontent.com/eccho03/datamining_project_clothes/refs/heads/main/apriori_result.csv')
allowed_sizes = ['XL이상', 'L', 'M', 'S', 'XS이하']
size_data = size_data[size_data['size'].isin(allowed_sizes)]

In [7]:
size_data

,min_height,max_height,min_weight,max_weight,size,confidence
0,165.0,169.0,80.0,inf,XL이상,0.993502
1,NaN,NaN,80.0,inf,XL이상,0.988845
2,170.0,174.0,80.0,inf,XL이상,0.985016
3,NaN,NaN,75.0,79.0,XL이상,0.963593
4,160.0,164.0,70.0,74.0,XL이상,0.918229
...,...,...,...,...,...,...
200,170.0,174.0,NaN,NaN,L,0.050168
203,160.0,164.0,NaN,NaN,XS이하,0.049284
204,165.0,169.0,NaN,NaN,XL이상,0.049140
206,NaN,NaN,55.0,59.0,L,0.047696


In [8]:
data

,page_url,제품명,제품 소개,cate1,cate2,가격,성별,XS이하,S,M,L,XL이상
0,https://www.nike.com/kr/t/스포츠웨어-칠-니트-여성-슬림-긴팔-...,여성 슬림 긴팔 크롭 탑,나이키 스포츠웨어 칠 니트,탑 & 티셔츠,그래픽 티셔츠,"55,000 원",W,1,1,1,1,1
1,https://www.nike.com/kr/t/스포츠웨어-여성-니트-긴팔-크롭-탑-...,여성 니트 긴팔 크롭 탑,나이키 스포츠웨어,탑 & 티셔츠,그래픽 티셔츠,"59,000 원",W,0,1,1,1,1
2,https://www.nike.com/kr/t/acg-여성-드라이-핏-루즈-반팔-티...,여성 드라이 핏 루즈 반팔 티셔츠,나이키 ACG,탑 & 티셔츠,그래픽 티셔츠,"59,000 원",W,0,1,1,1,1
3,https://www.nike.com/kr/t/프로-여성-드라이-핏-긴팔-크롭-티셔...,여성 드라이 핏 긴팔 크롭 티셔츠,나이키 프로,탑 & 티셔츠,그래픽 티셔츠,"55,000 원",W,0,0,0,0,1
4,https://www.nike.com/kr/t/조던-헤리티지-여성-그래픽-티셔츠-T...,여성 그래픽 티셔츠,조던 헤리티지,탑 & 티셔츠,그래픽 티셔츠,"59,000 원",W,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
371,https://www.nike.com/kr/t/드라이-핏-어드벤티지-여성-롱-골프-...,여성 롱 골프 스커트,나이키 드라이 핏 어드벤티지,스커트 & 드레스,스커트 & 드레스,"119,000 원",W,0,1,0,0,0
372,https://www.nike.com/kr/t/원-여성-드라이-핏-드레스-ZpwzN...,여성 드라이 핏 드레스,나이키 원,스커트 & 드레스,스커트 & 드레스,"95,000 원",W,0,0,0,1,0
373,https://www.nike.com/kr/t/조던-여성-슬림-니트-드레스-9uMf...,여성 슬림 니트 드레스,조던,스커트 & 드레스,스커트 & 드레스,"75,000 원",W,0,1,1,1,1
374,https://www.nike.com/kr/t/코트-드라이-핏-빅토리-여성-플로운시...,여성 플로운시 테니스 스커트,나이키코트 드라이 핏 빅토리,스커트 & 드레스,스커트 & 드레스,"69,000 원",W,0,1,1,1,1


In [9]:
def parse_price(price_str):
    return int(re.sub(r'[^0-9]', '', price_str))  # 기존가격 데이터(문자열 -> 숫자)

data['가격'] = data['가격'].apply(parse_price)

# 사이즈 도출

In [10]:
# 키와 몸무게로부터 사이즈 도출 함수
def get_size_from_height_weight(height, weight):
    size_rows = size_data[
        ((size_data['min_height'].isna()) | (size_data['min_height'] <= height)) &
        ((size_data['max_height'].isna()) | (size_data['max_height'] >= height)) &
        ((size_data['min_weight'].isna()) | (size_data['min_weight'] <= weight)) &
        ((size_data['max_weight'].isna()) | (size_data['max_weight'] >= weight))
    ]
    if not size_rows.empty:
        # Confidence가 가장 높은 사이즈 선택
        best_size_row = size_rows.loc[size_rows['confidence'].idxmax()]
        return best_size_row['size']
    else:
        return None

In [11]:
user_size = get_size_from_height_weight(user_height, user_weight)

In [12]:
user_size

'S'

# item 기반 협업 필터링

In [13]:
from IPython.display import display

In [14]:
import re
from urllib.parse import quote

# 원본 URL 문자열
url = "https://www.nike.com/kr/t/조던-브루클린-플리스-남성-크루넥-스"

# 한글을 찾는 정규식 패턴
def encode_korean(url):
    return re.sub(r'[\uac00-\ud7a3]+', lambda match: quote(match.group(0)), url)

In [15]:
if user_size is None:
    print("사용자에게 맞는 사이즈가 없습니다.")
else:
    # 사용자가 선택한 아이템의 index를 입력받음
    selected_item_index = select_clothes
    selected_item = data.loc[selected_item_index]

    # 사이즈와 가격 범위 필터링: 사용자의 사이즈와 맞는 아이템 필터링
    filtered_items = data[(data[user_size] == 1) & (data['가격'] >= min_price) & (data['가격'] <= max_price)].copy()

    # 필터링된 데이터 개수 출력
    filtered_count = len(filtered_items)
    print(f"사용자 사이즈 '{user_size}' 및 가격 범위 {min_price}-{max_price}에 맞는 아이템 개수: {filtered_count}")

    # 아이템 리스트에 선택한 아이템 추가
    filtered_items = pd.concat([selected_item.to_frame().T, filtered_items], ignore_index=True)

    # 아이템 속성을 활용하여 아이템 간 유사도 계산 (아이템 기반 협업 필터링)
    df_features = pd.get_dummies(filtered_items[['가격', 'cate1', 'cate2']])
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_features)

    # 코사인 유사도를 사용해 아이템 간 유사도 계산
    item_similarity_matrix = cosine_similarity(scaled_features)

    # 선택한 아이템의 인덱스를 기준으로 유사도 계산
    item_similarities = item_similarity_matrix[filtered_items.index.get_loc(selected_item_index)]

    # 유사도 기준으로 상위 5개 아이템 추천
    top_n = 5  # 추천할 아이템 개수
    recommended_indices = np.argsort(item_similarities)[::-1][1:top_n+1]  # 자기 자신 제외하고 상위 N개
    recommended_items = filtered_items.iloc[recommended_indices]

    # 추천 결과 출력 (사용자가 선택한 옷이 사이즈가 없는 경우 추천 목록에서 제외)
    recommended_items = recommended_items[(recommended_items[user_size] == 1) & (recommended_items['가격'] >= min_price) & (recommended_items['가격'] <= max_price)]

    pd.set_option('display.max_colwidth', None)
    recommended_items['page_url'] = recommended_items['page_url'].apply(encode_korean)
    print("사용자가 선호한 아이템:")
    display(selected_item[['page_url', '제품명', 'cate1', 'cate2', '가격']])
    print("사용자에게 추천할 아이템들:")
    display(recommended_items[['page_url', '제품명', 'cate1', 'cate2', '가격']])

사용자 사이즈 'S' 및 가격 범위 50000-300000에 맞는 아이템 개수: 228
사용자가 선호한 아이템:


,50
page_url,https://www.nike.com/kr/t/드라이-핏-uv-어드밴티지-여성-풀집-골프-탑-khzF7XgZ/DX1490-675
제품명,여성 풀집 골프 탑
cate1,탑 & 티셔츠
cate2,긴팔
가격,129000


사용자에게 추천할 아이템들:


,page_url,제품명,cate1,cate2,가격
61,https://www.nike.com/kr/t/%EC%8A%A4%ED%8F%AC%EC%B8%A0%EC%9B%A8%EC%96%B4-%EC%97%AC%EC%84%B1-%ED%8B%B0%EC%85%94%EC%B8%A0-Vsz31Xnc/HQ4327-685,여성 티셔츠,탑 & 티셔츠,반팔,55000
52,https://www.nike.com/kr/t/sb-%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%8A%B8%EB%B3%B4%EB%94%A9-%ED%8B%B0%EC%85%94%EC%B8%A0-9iuZr3Hz/FZ4109-100,스케이트보딩 티셔츠,탑 & 티셔츠,반팔,55000
50,https://www.nike.com/kr/t/%EC%9B%90-%ED%94%BC%ED%8B%B0%EB%93%9C-%EC%97%AC%EC%84%B1-%EB%93%9C%EB%9D%BC%EC%9D%B4-%ED%95%8F-%EB%B0%98%ED%8C%94-%ED%81%AC%EB%A1%AD-%ED%83%91-pcNxzUxj/FN2805-010,여성 드라이 핏 반팔 크롭 탑,탑 & 티셔츠,반팔,55000
15,https://www.nike.com/kr/t/%EC%8A%A4%ED%8F%AC%EC%B8%A0%EC%9B%A8%EC%96%B4-%EC%B9%A0-%EB%8B%88%ED%8A%B8-%EC%97%AC%EC%84%B1-%EC%8A%AC%EB%A6%BC-%EA%B8%B4%ED%8C%94-%ED%81%AC%EB%A1%AD-%ED%83%91-MVpJND1j/HF5323-338,여성 슬림 긴팔 크롭 탑,탑 & 티셔츠,긴팔,55000
32,https://www.nike.com/kr/t/%EC%9B%90-%ED%81%B4%EB%9E%98%EC%8B%9D-%EC%97%AC%EC%84%B1-%EB%93%9C%EB%9D%BC%EC%9D%B4-%ED%95%8F-%EA%B8%B4%ED%8C%94-%ED%83%91-KielCNTG/FN2802-010,여성 드라이 핏 긴팔 탑,탑 & 티셔츠,긴팔,55000
